# M06: Advanced Transformations

The largest exam domain (29%). We cover advanced transformations in PySpark and SQL: window functions (ranking, lag/lead), array operations (explode), JSON processing, CTEs, subqueries, CASE WHEN, UDFs, and higher-order functions. Each technique is demonstrated in both PySpark and SQL.

| Exam Domain | Weight |
|---|---|
| ELT with Spark SQL and Python | 29% |
---

## Setup

Initialize the environment, import required PySpark functions, and configure catalog/schema references for this module.

---

In [0]:
%run ../../setup/00_setup

### Configuration

Import all required PySpark modules (window, functions, types) and set the active catalog and schema.

In [0]:
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, row_number, rank, dense_rank, lag, lead,
    sum as _sum, avg, count, max as _max, min as _min,
    to_date, date_trunc, date_add, add_months, last_day,
    explode, posexplode, sequence, from_json, to_json, schema_of_json,
    current_timestamp, round as _round, lit, when, coalesce,
    udf, array, transform, filter, exists
)
from pyspark.sql.types import *
import datetime

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

## Part A: PySpark Transformations

Deep dive into PySpark's advanced transformation capabilities — window functions (ranking, lag/lead, rolling aggregations), complex types (arrays with explode/posexplode), JSON parsing, and date/time functions for temporal analysis.

---

### Window Functions — Basics

| Element | Purpose |
|---|---|
| `partitionBy()` | Dividing data into groups |
| `orderBy()` | Sorting within a partition |
| `rowsBetween()` / `rangeBetween()` | Defining the window frame |

**Use cases:** Ranking, time comparisons (lag/lead), moving aggregations, trend analysis

In [0]:
# Prepare sample order data
orders_data = [
    (1, 1, "2024-01-15", 150.0),
    (2, 2, "2024-01-16", 200.0),
    (3, 1, "2024-02-10", 300.0),
    (4, 3, "2024-02-12", 100.0),
    (5, 2, "2024-03-05", 450.0),
    (6, 1, "2024-03-15", 250.0),
    (7, 3, "2024-03-20", 180.0),
    (8, 2, "2024-04-01", 320.0),
    (9, 1, "2024-04-10", 400.0),
    (10, 3, "2024-04-15", 220.0),
]

In [0]:
orders_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("amount", DoubleType(), False)
])

In [0]:
orders_df = spark.createDataFrame(orders_data, orders_schema)
orders_df = orders_df.withColumn("order_date", to_date(col("order_date")))
orders_df.display()

### Demo: Ranking — row_number, rank, dense_rank

- `row_number()`: Unique numbers (1, 2, 3, 4, ...)
- `rank()`: With gaps for ties (1, 2, 2, 4, ...)
- `dense_rank()`: Without gaps for ties (1, 2, 2, 3, ...)

In [0]:
# Ranking orders for each customer by amount (descending)
window_spec = Window.partitionBy("customer_id").orderBy(col("amount").desc())

orders_ranked = orders_df.withColumn("row_num", row_number().over(window_spec)) \
    .withColumn("rank", rank().over(window_spec)) \
    .withColumn("dense_rank", dense_rank().over(window_spec))

In [0]:
orders_ranked.orderBy("customer_id", "row_num").display()

### Demo: Lag and Lead Functions

- **`lag()`** — value from previous row (e.g., comparison with previous period)
- **`lead()`** — value from next row (e.g., calculating deltas)

In [0]:
# Comparison with previous and next order
window_spec_time = Window.partitionBy("customer_id").orderBy("order_date")

orders_lag_lead = orders_df \
    .withColumn("prev_amount", lag("amount", 1).over(window_spec_time)) \
    .withColumn("next_amount", lead("amount", 1).over(window_spec_time)) \
    .withColumn("amount_change", col("amount") - col("prev_amount")) \
    .withColumn("amount_change_pct", 
                _round((col("amount") - col("prev_amount")) / col("prev_amount") * 100, 2))

In [0]:
orders_lag_lead.orderBy("customer_id", "order_date").display()

### Rolling Windows — Moving Aggregations

| Method | Description |
|---|---|
| `rowsBetween(start, end)` | Row-based range |
| `rangeBetween(start, end)` | Value-based range (requires `orderBy`) |
| `Window.unboundedPreceding` | From start of partition |
| `Window.unboundedFollowing` | To end of partition |
| `-2, 0` | Two previous + current (3-row window) |

In [0]:
# Rolling sum (all previous + current)
window_cumulative = Window.partitionBy("customer_id") \
    .orderBy("order_date") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

orders_cumulative = orders_df \
    .withColumn("cumulative_sum", _sum("amount").over(window_cumulative)) \
    .withColumn("cumulative_count", count("order_id").over(window_cumulative)) \
    .withColumn("cumulative_avg", _round(avg("amount").over(window_cumulative), 2))

In [0]:
orders_cumulative.orderBy("customer_id", "order_date").display()

In [0]:
# Moving average - 3 last orders
window_moving_3 = Window.partitionBy("customer_id") \
    .orderBy("order_date") \
    .rowsBetween(-2, 0)  # 2 previous + current = 3 rows

orders_moving_avg = orders_df \
    .withColumn("moving_avg_3", _round(avg("amount").over(window_moving_3), 2)) \
    .withColumn("moving_sum_3", _sum("amount").over(window_moving_3)) \
    .withColumn("moving_max_3", _max("amount").over(window_moving_3)) \
    .withColumn("moving_min_3", _min("amount").over(window_moving_3))

### Calculating Moving Average

Applying moving window to calculate:
- **moving_avg_3**: Average of last 3 orders
- **moving_sum_3**: Sum of last 3 orders
- **moving_max_3**: Max of last 3 orders
- **moving_min_3**: Min of last 3 orders

**rowsBetween(-2, 0)** means: 2 previous rows + current = 3-row window

In [0]:
orders_moving_avg.orderBy("customer_id", "order_date").display()

### Complex Types — Arrays

- **`explode()`** — array → separate rows
- **`posexplode()`** — like explode, but with position index

**Use cases:** Normalizing nested data, list analysis (tags, products), event tracking

In [0]:
# Example: Products in orders (array)
orders_with_products_data = [
    (1, 1, "2024-01-15", ["Laptop", "Mouse", "Keyboard"]),
    (2, 2, "2024-01-16", ["Monitor", "Cable"]),
    (3, 1, "2024-02-10", ["Headphones"]),
    (4, 3, "2024-02-12", ["Tablet", "Case", "Stylus", "Charger"]),
]

**Original Data (with arrays)**

Each order contains a list of products as an array. Data structure:
- `order_id`: Order ID
- `customer_id`: Customer ID
- `order_date`: Order Date
- `products`: Array with product names

In [0]:
orders_products_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("products", ArrayType(StringType()), False)
])

In [0]:
orders_products_df = spark.createDataFrame(orders_with_products_data, orders_products_schema)
display(orders_products_df.limit(5))

In [0]:
# explode() - explodes array into separate rows
orders_exploded = orders_products_df.select(
    "order_id",
    "customer_id",
    "order_date",
    explode("products").alias("product")
)
orders_exploded.display()

In [0]:
orders_posexploded = orders_products_df.select(
    "order_id",
    "customer_id",
    "order_date",
    posexplode("products").alias("position", "product")
)

In [0]:
orders_posexploded.display()

### Demo: Shopping Basket Analysis

**Scenario:** Orders with product lists — analyze co-purchases, basket size, top products per category

In [0]:
# Data: Orders with products, prices, and categories
basket_data = [
    (101, 1, "Premium", "2024-01-15", [
        {"product": "Laptop", "price": 1200.0, "category": "Electronics"},
        {"product": "Mouse", "price": 25.0, "category": "Accessories"},
        {"product": "Keyboard", "price": 75.0, "category": "Accessories"},
        {"product": "USB Cable", "price": 10.0, "category": "Accessories"}
    ]),
    (102, 2, "Standard", "2024-01-16", [
        {"product": "Monitor", "price": 300.0, "category": "Electronics"},
        {"product": "HDMI Cable", "price": 15.0, "category": "Accessories"}
    ]),
    (103, 1, "Premium", "2024-02-10", [
        {"product": "Headphones", "price": 150.0, "category": "Audio"},
        {"product": "Laptop", "price": 1200.0, "category": "Electronics"}
    ]),
    (104, 3, "Standard", "2024-02-12", [
        {"product": "Tablet", "price": 400.0, "category": "Electronics"},
        {"product": "Case", "price": 30.0, "category": "Accessories"},
        {"product": "Stylus", "price": 50.0, "category": "Accessories"},
        {"product": "Charger", "price": 25.0, "category": "Accessories"}
    ]),
    (105, 2, "Premium", "2024-03-05", [
        {"product": "Smartphone", "price": 800.0, "category": "Electronics"},
        {"product": "Screen Protector", "price": 20.0, "category": "Accessories"},
        {"product": "Phone Case", "price": 35.0, "category": "Accessories"}
    ]),
]

In [0]:
# Schema for nested data
basket_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("customer_tier", StringType(), False),
    StructField("order_date", StringType(), False),
    StructField("items", ArrayType(
        StructType([
            StructField("product", StringType(), False),
            StructField("price", DoubleType(), False),
            StructField("category", StringType(), False)
        ])
    ), False)
])

In [0]:
basket_df = spark.createDataFrame(basket_data, basket_schema) \
    .withColumn("order_date", to_date(col("order_date")))

display(basket_df.limit(5))

**Original Data - Orders with Nested Products**

Each order contains:
- `order_id`: Order ID
- `customer_id`: Customer ID  
- `customer_tier`: Customer Tier (Premium/Standard)
- `order_date`: Order Date
- `items`: Array of structs with products (name, price, category)

In [0]:
# explode() - expanding nested products
basket_exploded = basket_df.select(
    "order_id",
    "customer_id",
    "customer_tier",
    "order_date",
    explode("items").alias("item")
).select(
    "order_id",
    "customer_id",
    "customer_tier",
    "order_date",
    col("item.product").alias("product"),
    col("item.price").alias("price"),
    col("item.category").alias("category")
)
basket_exploded.display()

In [0]:
# Summary: record count before and after explode
orders_before = basket_df.count()
items_after = basket_exploded.count()

display(spark.createDataFrame([
    ("Orders (before explode)", orders_before),
    ("Product items (after explode)", items_after)
], ["Metric", "Value"]))

**Result:** Data after explode - each product in a separate row

**Transformation Statistics:**
- Orders (before explode): 5
- Product items (after explode): ~15+ (depending on number of products in each order)

In [0]:
# Analysis 1: Top 5 Most Popular Products
top_products = basket_exploded \
    .groupBy("product", "category") \
    .agg(
        count("*").alias("times_ordered"),
        _sum("price").alias("total_revenue"),
        _round(avg("price"), 2).alias("avg_price")
    ) \
    .orderBy(col("times_ordered").desc()) \
    .limit(5)
top_products.display()

**TOP 5 Most Popular Products**

Analysis shows which products are most frequently ordered, along with total revenue and average price.

In [0]:
# Analysis 2: Sales per Category
category_sales = basket_exploded \
    .groupBy("category") \
    .agg(
        count("*").alias("items_sold"),
        _sum("price").alias("revenue"),
        _round(avg("price"), 2).alias("avg_item_price")
    ) \
    .orderBy(col("revenue").desc())
category_sales.display()

**Sales per Category**

Aggregated sales statistics by product category.

In [0]:
from pyspark.sql.functions import when

# Analysis 3: Shopping Basket Metrics per Order
basket_metrics = basket_exploded \
    .groupBy("order_id", "customer_id", "customer_tier", "order_date") \
    .agg(
        count("product").alias("basket_size"),
        _sum("price").alias("order_total"),
        _round(avg("price"), 2).alias("avg_item_price"),
        _max("price").alias("most_expensive_item"),
        count(when(col("category") == "Electronics", 1)).alias("electronics_count"),
        count(when(col("category") == "Accessories", 1)).alias("accessories_count")
    ) \
    .orderBy("order_id")
basket_metrics.display()

**Shopping Basket Metrics**

Analysis of each order regarding:
- Basket size (number of products)
- Total order value
- Average product price in basket
- Most expensive product
- Category breakdown (Electronics vs Accessories)

In [0]:
# Summary per Customer Tier
tier_analysis = basket_metrics \
    .groupBy("customer_tier") \
    .agg(
        count("order_id").alias("orders_count"),
        _round(avg("basket_size"), 2).alias("avg_basket_size"),
        _round(avg("order_total"), 2).alias("avg_order_value"),
        _sum("order_total").alias("total_revenue")
    ) \
    .orderBy(col("total_revenue").desc())
tier_analysis.display()

**Customer Tier Analysis**

Comparison of purchasing behavior between customer segments (Premium vs Standard).

In [0]:
# Analysis 4: Market Basket Analysis - Products Bought Together

# Preparation: Self-join of products from the same order
basket_pairs = basket_exploded.alias("a") \
    .join(
        basket_exploded.alias("b"),
        (col("a.order_id") == col("b.order_id")) & 
        (col("a.product") < col("b.product"))  # Avoiding duplicates (A+B = B+A)
    ) \
    .select(
        col("a.product").alias("product_a"),
        col("b.product").alias("product_b")
    )

# Count most frequent product pairs
product_pairs_count = basket_pairs \
    .groupBy("product_a", "product_b") \
    .agg(count("*").alias("times_together")) \
    .orderBy(col("times_together").desc())
product_pairs_count.display()

**Products Bought Together (Market Basket Analysis)**

Self-join of the same order to find all pairs of products bought together. The condition `product_a < product_b` eliminates duplicates.

In [0]:
# Insight: Which products are most frequently bought with Electronics?
electronics_combos = basket_exploded.alias("e") \
    .join(
        basket_exploded.alias("a"),
        (col("e.order_id") == col("a.order_id")) & 
        (col("e.category") == "Electronics") &
        (col("a.category") != "Electronics")
    ) \
    .groupBy(col("e.product").alias("electronics_item"), 
             col("a.product").alias("paired_with")) \
    .agg(count("*").alias("combo_count")) \
    .orderBy(col("combo_count").desc()) \
    .limit(10)
electronics_combos.display()

**Products Most Frequently Bought with Electronics**

Cross-selling analysis: which products from other categories are most frequently bought together with Electronics products.

### Key Takeaways from Example

**What we did:**
1. Used `explode()` to expand nested products (array of structs)
2. Calculated metrics per product and category
3. Analyzed shopping baskets (basket size, avg value)
4. Performed Market Basket Analysis (products bought together)

**Business Applications:**
- **Product Recommendations**: Products frequently bought together
- **Basket Analysis**: Average value, size per customer segment
- **Cross-selling**: Which accessories to sell with electronics
- **Revenue Optimization**: Categories generating the most revenue

**Performance Tips:**
- `explode()` increases row count - use with filter where possible
- Aggregate data after explode to reduce size
- For very large arrays consider partitioning before explode

### Demo: sequence() — Generating Sequences

`sequence(start, stop, step)` — generates an array of values (date ranges, time series, filling gaps)

In [0]:
# Example: Generating sequence of days between dates
from pyspark.sql.functions import expr

date_ranges_data = [
    ("2024-01-01", "2024-01-05"),
    ("2024-02-01", "2024-02-03"),
]

date_ranges_df = spark.createDataFrame(date_ranges_data, ["start_date", "end_date"]) \
    .withColumn("start_date", to_date(col("start_date"))) \
    .withColumn("end_date", to_date(col("end_date")))

In [0]:
# Generate sequence of days
date_sequence = date_ranges_df.withColumn(
    "date_array",
    expr("sequence(start_date, end_date, interval 1 day)")
)
display(date_sequence)

**Date sequence as array**

The `sequence()` function creates an array of dates between `start_date` and `end_date` with a step of 1 day.

In [0]:
# Explode to separate rows
date_sequence.select(
    "start_date",
    "end_date",
    explode("date_array").alias("date")
).display()

**Date sequence after explode**

Each date from the array is exploded into a separate row - useful for time series analysis.

### JSON Processing

| Function | Purpose |
|---|---|
| `from_json()` | Parse JSON string → struct/array |
| `to_json()` | Convert struct/array → JSON string |
| `schema_of_json()` | Auto-detect JSON schema |

**Use cases:** API responses, event tracking, log processing

In [0]:
# Example: Parsing JSON payload
json_data = [
    (1, '{"user_id": 101, "action": "click", "metadata": {"page": "home", "duration": 30}}'),
    (2, '{"user_id": 102, "action": "purchase", "metadata": {"page": "checkout", "duration": 120}}'),
    (3, '{"user_id": 101, "action": "view", "metadata": {"page": "product", "duration": 45}}'),
]

In [0]:
json_df = spark.createDataFrame(json_data, ["event_id", "json_payload"])

**Original Data (JSON as string)**

Event data contains JSON payload as a string with nested structure:
- `user_id`: User ID
- `action`: Action type (click, purchase, view)
- `metadata`: Nested data (page, duration)

In [0]:
# Automatic JSON schema detection
json_schema = schema_of_json(lit(json_data[0][1]))

# Parsing JSON
json_parsed = json_df.withColumn("parsed_data", from_json(col("json_payload"), json_schema))
json_parsed.display()

**Automatic schema detection and JSON parsing**

1. `schema_of_json()` automatically detects schema from sample JSON
2. `from_json()` parses JSON string to struct according to detected schema

In [0]:
# Extracting fields from nested JSON
json_flattened = json_parsed.select(
    "event_id",
    col("parsed_data.user_id").alias("user_id"),
    col("parsed_data.action").alias("action"),
    col("parsed_data.metadata.page").alias("page"),
    col("parsed_data.metadata.duration").alias("duration")
)
json_flattened.display()

**Flattened Data**

Extracting specific fields from nested JSON structure:
- Accessing top-level fields: `parsed_data.user_id`
- Accessing nested fields: `parsed_data.metadata.page`

### Date and Time Functions

| Function | Purpose |
|---|---|
| `date_trunc()` | Truncate to boundary (year, month, day, hour) |
| `date_add()` / `add_months()` | Adding days / months |
| `last_day()` | Last day of month |
| `datediff()` / `months_between()` | Difference in days / months |

**Use cases:** Temporal aggregations, cohort analysis, retention analysis

In [0]:
from pyspark.sql.functions import datediff, months_between
from pyspark.sql.functions import year, month, dayofweek, quarter

# Example: Temporal Order Analysis
orders_temporal = orders_df \
    .withColumn("year", year("order_date")) \
    .withColumn("month", month("order_date")) \
    .withColumn("quarter", quarter("order_date")) \
    .withColumn("day_of_week", dayofweek("order_date")) \
    .withColumn("month_start", date_trunc("month", "order_date")) \
    .withColumn("month_end", last_day("order_date")) \
    .withColumn("next_month_start", date_add(last_day("order_date"), 1))
orders_temporal.display()

### Temporal Analysis Result

For each order we calculate:
- **year/month/quarter**: Time components of order date
- **day_of_week**: Day of week (1=Sunday, 7=Saturday)
- **month_start**: First day of month
- **month_end**: Last day of month
- **next_month_start**: First day of next month

**Business Applications:**
- Sales seasonality analysis
- Reporting per period (monthly, quarterly)
- Trend analysis by day of week

In [0]:
# Example: Calculating periods between orders
window_date = Window.partitionBy("customer_id").orderBy("order_date")

orders_periods = orders_df \
    .withColumn("prev_order_date", lag("order_date", 1).over(window_date)) \
    .withColumn("days_since_last_order", 
                datediff(col("order_date"), col("prev_order_date"))) \
    .withColumn("months_since_last_order", 
                _round(months_between(col("order_date"), col("prev_order_date")), 2))

In [0]:
orders_periods.orderBy("customer_id", "order_date").display()

### Analysis of periods between orders

**Calculated metrics:**
- **prev_order_date**: Previous order date (using `lag`)
- **days_since_last_order**: Number of days since last order (`datediff`)
- **months_since_last_order**: Number of months since last order (`months_between`)

**Business Applications:**
- Customer purchase frequency analysis
- Identifying dormant customers (long periods without orders)
- Customer segmentation by frequency
- Next order prediction

### Demo: Customer Behavior Analysis

Comprehensive analysis combining: order ranking, lag comparison, moving average, temporal segmentation

In [0]:
# Defining windows for analysis
window_customer_time = Window.partitionBy("customer_id").orderBy("order_date")
window_customer_amount = Window.partitionBy("customer_id").orderBy(col("amount").desc())
window_moving_avg = Window.partitionBy("customer_id").orderBy("order_date").rowsBetween(-2, 0)
window_cumulative = Window.partitionBy("customer_id").orderBy("order_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Comprehensive customer behavior analysis
customer_behavior = orders_df \
    .withColumn("order_rank", row_number().over(window_customer_amount)) \
    .withColumn("order_sequence", row_number().over(window_customer_time)) \
    .withColumn("prev_amount", lag("amount", 1).over(window_customer_time)) \
    .withColumn("amount_change", col("amount") - col("prev_amount")) \
    .withColumn("moving_avg_3", _round(avg("amount").over(window_moving_avg), 2)) \
    .withColumn("cumulative_spent", _sum("amount").over(window_cumulative)) \
    .withColumn("month", date_trunc("month", "order_date"))

In [0]:
customer_behavior.orderBy("customer_id", "order_date").display()

## Part B: SQL Transformations

Explore the same transformation patterns using Spark SQL — including temp views, JOINs, window functions, CTEs, subqueries (scalar, correlated, EXISTS/IN), CASE WHEN expressions, and DDL operations for persisting results.

---

### Spark SQL Basics

`spark.sql()` executes SQL and returns a DataFrame. Supports all standard SQL, uses **Catalyst Optimizer**.

In [0]:
# Example: Simple SQL query
result = spark.sql("""
    SELECT 
        'Hello Spark SQL' as message,
        current_date() as today,
        current_timestamp() as now
""")
display(result)

### Creating Test Data

Preparing data for Spark SQL demonstration:

In [0]:
# Orders data
orders_data = [
    (1, 101, "2024-01-15", 250.00, "completed"),
    (2, 102, "2024-01-16", 150.00, "completed"),
    (3, 101, "2024-01-20", 320.00, "completed"),
    (4, 103, "2024-02-01", 180.00, "pending"),
    (5, 101, "2024-02-10", 420.00, "completed"),
    (6, 102, "2024-02-15", 90.00, "cancelled"),
    (7, 103, "2024-03-01", 550.00, "completed"),
    (8, 104, "2024-03-05", 280.00, "completed"),
    (9, 101, "2024-03-10", 175.00, "completed"),
    (10, 102, "2024-03-15", 340.00, "completed"),
]

orders_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("amount", DoubleType(), False),
    StructField("status", StringType(), False)
])

orders_df = spark.createDataFrame(orders_data, orders_schema) \
    .withColumn("order_date", F.to_date("order_date"))

In [0]:
# Customers data
customers_data = [
    (101, "Jan", "Kowalski", "Premium", "Warszawa"),
    (102, "Anna", "Nowak", "Standard", "Krakow"),
    (103, "Piotr", "Wisniewski", "Premium", "Gdansk"),
    (104, "Maria", "Wojcik", "Standard", "Poznan"),
]

customers_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), False),
    StructField("tier", StringType(), False),
    StructField("city", StringType(), False)
])

customers_df = spark.createDataFrame(customers_data, customers_schema)
display(orders_df.limit(5))
display(customers_df)

### Registering Temp Views

To use DataFrames in SQL queries, they must be registered as temporary views.

**View types:**
- `createOrReplaceTempView()` - local view for the session
- `createOrReplaceGlobalTempView()` - global view (accessible via `global_temp.name`)

In [0]:
# Registering temporary views
orders_df.createOrReplaceTempView("orders")
customers_df.createOrReplaceTempView("customers")

In [0]:
# Now we can use SQL
spark.sql("SELECT * FROM orders LIMIT 5").display()

### SQL vs DataFrame API Comparison

Side-by-side comparison showing that DataFrame API and Spark SQL produce identical results and execution plans.

**Task:** Find total value of completed orders per customer

In [0]:
# DataFrame API Approach
result_df = orders_df \
    .filter(F.col("status") == "completed") \
    .groupBy("customer_id") \
    .agg(
        F.count("*").alias("orders_count"),
        F.sum("amount").alias("total_amount"),
        F.round(F.avg("amount"), 2).alias("avg_amount")
    ) \
    .orderBy(F.col("total_amount").desc())
display(result_df)

In [0]:
# Spark SQL Approach
result_sql = spark.sql("""
    SELECT 
        customer_id,
        COUNT(*) as orders_count,
        SUM(amount) as total_amount,
        ROUND(AVG(amount), 2) as avg_amount
    FROM orders
    WHERE status = 'completed'
    GROUP BY customer_id
    ORDER BY total_amount DESC
""")
display(result_sql)

**Comparison:** Both approaches yield identical results and execution plans.

### Example: JOIN with multiple tables

In [0]:
# DataFrame API - JOIN
joined_df = orders_df \
    .join(customers_df, "customer_id", "inner") \
    .select(
        "order_id",
        F.concat_ws(" ", "first_name", "last_name").alias("customer_name"),
        "tier",
        "order_date",
        "amount",
        "status"
    )
display(joined_df.limit(5))

In [0]:
# Spark SQL - JOIN
joined_sql = spark.sql("""
    SELECT 
        o.order_id,
        CONCAT_WS(' ', c.first_name, c.last_name) as customer_name,
        c.tier,
        o.order_date,
        o.amount,
        o.status
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
""")
display(joined_sql.limit(5))

### Window Functions in SQL

```sql
function() OVER (PARTITION BY col ORDER BY col ROWS BETWEEN ... AND ...)
```

| Type | Functions |
|---|---|
| Ranking | `ROW_NUMBER()`, `RANK()`, `DENSE_RANK()` |
| Analytic | `LAG()`, `LEAD()`, `FIRST_VALUE()`, `LAST_VALUE()` |

In [0]:
# Order ranking per customer
ranking_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        order_date,
        amount,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) as order_sequence,
        RANK() OVER (PARTITION BY customer_id ORDER BY amount DESC) as amount_rank,
        DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY amount DESC) as amount_dense_rank
    FROM orders
    WHERE status = 'completed'
    ORDER BY customer_id, order_date
""")
display(ranking_sql)

### LAG and LEAD - Change Analysis

Using `LAG()` and `LEAD()` in SQL to compare each order with the previous/next one and calculate amount differences.

In [0]:
# Comparison with previous order
lag_lead_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        order_date,
        amount,
        LAG(amount, 1) OVER (PARTITION BY customer_id ORDER BY order_date) as prev_amount,
        LEAD(amount, 1) OVER (PARTITION BY customer_id ORDER BY order_date) as next_amount,
        amount - LAG(amount, 1) OVER (PARTITION BY customer_id ORDER BY order_date) as amount_change
    FROM orders
    WHERE status = 'completed'
    ORDER BY customer_id, order_date
""")
display(lag_lead_sql)

### Running Totals and Moving Averages

Cumulative sums and sliding-window averages using SQL `ROWS BETWEEN` frame specifications.

In [0]:
# Running total and moving average
running_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        order_date,
        amount,
        SUM(amount) OVER (
            PARTITION BY customer_id 
            ORDER BY order_date 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) as cumulative_amount,
        ROUND(AVG(amount) OVER (
            PARTITION BY customer_id 
            ORDER BY order_date 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ), 2) as moving_avg_3
    FROM orders
    WHERE status = 'completed'
    ORDER BY customer_id, order_date
""")
display(running_sql)

### CTE (Common Table Expressions)

`WITH` clause — named subqueries, reusable, readable, easier to debug.

> ~~Recursive CTEs~~ not supported in Spark SQL.

In [0]:
# CTE - Customer Analysis
cte_analysis = spark.sql("""
    WITH customer_orders AS (
        SELECT 
            customer_id,
            COUNT(*) as orders_count,
            SUM(amount) as total_spent,
            AVG(amount) as avg_order_value
        FROM orders
        WHERE status = 'completed'
        GROUP BY customer_id
    ),
    customer_ranking AS (
        SELECT 
            *,
            RANK() OVER (ORDER BY total_spent DESC) as spending_rank,
            CASE 
                WHEN total_spent >= 500 THEN 'High Value'
                WHEN total_spent >= 300 THEN 'Medium Value'
                ELSE 'Low Value'
            END as value_segment
        FROM customer_orders
    )
    SELECT 
        cr.*,
        c.first_name,
        c.last_name,
        c.tier,
        c.city
    FROM customer_ranking cr
    JOIN customers c ON cr.customer_id = c.customer_id
    ORDER BY spending_rank
""")
display(cte_analysis)

### Reusing CTEs

A single CTE can be referenced multiple times in the same query, avoiding redundant subqueries.

In [0]:
# CTE used multiple times
multi_cte = spark.sql("""
    WITH monthly_stats AS (
        SELECT 
            DATE_TRUNC('month', order_date) as month,
            customer_id,
            SUM(amount) as monthly_spent
        FROM orders
        WHERE status = 'completed'
        GROUP BY DATE_TRUNC('month', order_date), customer_id
    )
    SELECT 
        month,
        COUNT(DISTINCT customer_id) as active_customers,
        SUM(monthly_spent) as total_revenue,
        ROUND(AVG(monthly_spent), 2) as avg_customer_spend,
        MAX(monthly_spent) as max_customer_spend
    FROM monthly_stats
    GROUP BY month
    ORDER BY month
""")
display(multi_cte)

### Subqueries

Scalar subqueries embedded in SELECT and WHERE clauses to compare rows against aggregated values.

In [0]:
# Orders above average
scalar_subquery = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        amount,
        (SELECT ROUND(AVG(amount), 2) FROM orders WHERE status = 'completed') as avg_amount,
        amount - (SELECT AVG(amount) FROM orders WHERE status = 'completed') as diff_from_avg
    FROM orders
    WHERE status = 'completed'
      AND amount > (SELECT AVG(amount) FROM orders WHERE status = 'completed')
    ORDER BY amount DESC
""")
display(scalar_subquery)

### Correlated Subqueries

Subqueries referencing the outer query:

In [0]:
# Customers with orders above their average
correlated_subquery = spark.sql("""
    SELECT 
        o.order_id,
        o.customer_id,
        o.amount,
        (SELECT ROUND(AVG(o2.amount), 2) 
         FROM orders o2 
         WHERE o2.customer_id = o.customer_id 
           AND o2.status = 'completed') as customer_avg
    FROM orders o
    WHERE o.status = 'completed'
      AND o.amount > (
          SELECT AVG(o2.amount) 
          FROM orders o2 
          WHERE o2.customer_id = o.customer_id 
            AND o2.status = 'completed'
      )
    ORDER BY o.customer_id, o.amount DESC
""")
display(correlated_subquery)

### EXISTS and IN

Filtering rows based on the existence of related records in another table using `EXISTS` and `IN` operators.

In [0]:
# Customers who have orders > 400
exists_query = spark.sql("""
    SELECT 
        c.customer_id,
        c.first_name,
        c.last_name,
        c.tier
    FROM customers c
    WHERE EXISTS (
        SELECT 1 
        FROM orders o 
        WHERE o.customer_id = c.customer_id 
          AND o.amount > 400
          AND o.status = 'completed'
    )
""")
display(exists_query)

### CASE WHEN and Advanced Expressions

Conditional logic in SQL for order segmentation, status mapping, and safe value handling.

In [0]:
# Order segmentation
case_when_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        amount,
        CASE 
            WHEN amount >= 500 THEN 'Large'
            WHEN amount >= 200 THEN 'Medium'
            ELSE 'Small'
        END as order_size,
        CASE status
            WHEN 'completed' THEN 1
            WHEN 'pending' THEN 0
            ELSE -1
        END as status_code,
        COALESCE(amount, 0) as amount_safe
    FROM orders
    ORDER BY amount DESC
""")
display(case_when_sql)

### NULLIF, COALESCE, NVL

SQL functions for handling NULL values — replacing, defaulting, and conditionally nullifying columns.

In [0]:
# Handling NULLs
null_handling = spark.sql("""
    SELECT 
        order_id,
        amount,
        status,
        NULLIF(status, 'cancelled') as status_or_null,
        COALESCE(NULLIF(status, 'cancelled'), 'N/A') as status_clean,
        NVL(amount, 0) as amount_nvl
    FROM orders
""")
display(null_handling)

### DDL Operations

Using `CREATE OR REPLACE TABLE ... AS SELECT` (CTAS) to persist aggregated results into the gold schema.

In [0]:
# Creating table with aggregation results
spark.sql(f"""
    CREATE OR REPLACE TABLE {CATALOG}.{GOLD_SCHEMA}.customer_summary AS
    SELECT 
        c.customer_id,
        c.first_name,
        c.last_name,
        c.tier,
        c.city,
        COUNT(o.order_id) as total_orders,
        COALESCE(SUM(CASE WHEN o.status = 'completed' THEN o.amount END), 0) as total_spent,
        ROUND(COALESCE(AVG(CASE WHEN o.status = 'completed' THEN o.amount END), 0), 2) as avg_order_value,
        MAX(o.order_date) as last_order_date
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name, c.tier, c.city
""")

In [0]:
# Verification
spark.sql(f"SELECT * FROM {CATALOG}.{GOLD_SCHEMA}.customer_summary").display()

### CREATE VIEW

Creating reusable virtual tables (views) that encapsulate complex SQL logic for reporting.

In [0]:
%skip
%sql
CREATE OR REPLACE TABLE bronze.orders_demo
AS
select * from orders

In [0]:
# Creating a view
spark.sql(f"""
    CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.v_monthly_revenue AS
    SELECT 
        DATE_TRUNC('month', order_date) as month,
        COUNT(*) as orders_count,
        SUM(amount) as total_revenue,
        ROUND(AVG(amount), 2) as avg_order_value
    FROM orders --bronze.orders_demo
    WHERE status = 'completed'
    GROUP BY DATE_TRUNC('month', order_date)
""")

In [0]:
spark.sql(f"SELECT * FROM {CATALOG}.{GOLD_SCHEMA}.v_monthly_revenue ORDER BY month").display()

## Part C: User-Defined Functions (UDFs)

| Type | Language | Registration | Performance |
|---|---|---|---|
| Python UDF | Python | `@udf` decorator / `spark.udf.register()` | Slowest (serialization) |
| Pandas UDF | Python | `@pandas_udf` | Fast (Arrow) |
| SQL UDF | SQL | `CREATE FUNCTION` | Fast (no serialization) |

> **Exam Note:** Python UDFs = slow (row-by-row serialization). SQL UDFs = faster (Spark engine).

### Example: Python UDF

In [0]:
# Python UDF - classify order value
@udf("string")
def classify_order(amount):
    if amount is None:
        return "Unknown"
    elif amount > 500:
        return "High Value"
    elif amount > 100:
        return "Medium Value"
    else:
        return "Low Value"

# Apply UDF to DataFrame
df_orders = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.orders")
df_classified = df_orders.withColumn("order_class", classify_order(col("total_amount")))
display(df_classified.select("order_id", "total_amount", "order_class").limit(10))

### Example: SQL UDF

In [0]:
%%sql
-- SQL UDF - no serialization overhead
CREATE OR REPLACE FUNCTION classify_order_sql(amount DOUBLE)
RETURNS STRING
RETURN CASE
    WHEN amount IS NULL THEN 'Unknown'
    WHEN amount > 500 THEN 'High Value'
    WHEN amount > 100 THEN 'Medium Value'
    ELSE 'Low Value'
END;

In [0]:
%%sql
-- Use the SQL UDF
SELECT order_id, total_amount, classify_order_sql(total_amount) AS order_class
FROM orders
LIMIT 10;

## Part D: Higher-Order Functions

Operate on arrays **without** exploding them:

| Function | Purpose | Example |
|---|---|---|
| `TRANSFORM` | Apply to each element | `TRANSFORM(arr, x -> x * 2)` |
| `FILTER` | Keep matching elements | `FILTER(arr, x -> x > 0)` |
| `EXISTS` | Check if any matches | `EXISTS(arr, x -> x > 100)` |
| `AGGREGATE` | Reduce to single value | `AGGREGATE(arr, 0, (acc, x) -> acc + x)` |

> **Exam Note:** Higher-order functions > EXPLODE + GROUP BY for array operations.

### Example: TRANSFORM and FILTER in SQL

In [0]:
%%sql
-- Create sample data with arrays
CREATE OR REPLACE TEMP VIEW order_items AS
SELECT 1 AS order_id, array(10.0, 25.5, 8.0, 120.0) AS item_prices
UNION ALL
SELECT 2, array(5.0, 15.0, 200.0, 50.0)
UNION ALL
SELECT 3, array(99.0, 1.0, 45.0);

In [0]:
%%sql
-- TRANSFORM: apply 10% discount to all items
-- FILTER: keep only items above 20
-- EXISTS: check if any item is above 100
SELECT
    order_id,
    item_prices,
    TRANSFORM(item_prices, x -> ROUND(x * 0.9, 2)) AS discounted_prices,
    FILTER(item_prices, x -> x > 20) AS expensive_items,
    EXISTS(item_prices, x -> x > 100) AS has_premium_item
FROM order_items;

### Example: Higher-Order Functions in PySpark

In [0]:
from pyspark.sql.functions import transform, filter, exists, aggregate

df_items = spark.sql("SELECT * FROM order_items")

result = df_items.select(
    "order_id",
    "item_prices",
    transform("item_prices", lambda x: x * 0.9).alias("discounted"),
    filter("item_prices", lambda x: x > 20).alias("expensive"),
    exists("item_prices", lambda x: x > 100).alias("has_premium")
)
display(result)

## Summary

| Category | PySpark | SQL |
|---|---|---|
| Window Functions | `Window.partitionBy().orderBy()` | `OVER (PARTITION BY ... ORDER BY ...)` |
| Ranking | `row_number()`, `dense_rank()` | `ROW_NUMBER()`, `DENSE_RANK()` |
| Lag/Lead | `lag()`, `lead()` | `LAG()`, `LEAD()` |
| Complex Types | `explode()`, `posexplode()` | `EXPLODE()`, `LATERAL VIEW` |
| JSON | `from_json()`, `to_json()` | `from_json()`, `to_json()` |
| Date Functions | `date_trunc()`, `date_add()` | `DATE_TRUNC()`, `DATE_ADD()` |
| CTE | DataFrame chain | `WITH ... AS (...)` |
| Subqueries | `.filter(col.isin(...))` | `WHERE x IN (SELECT ...)` |
| Conditional | `when().otherwise()` | `CASE WHEN ... END` |
| UDFs | `@udf` decorator | `CREATE FUNCTION` |
| Higher-Order | `transform()`, `filter()` | `TRANSFORM()`, `FILTER()` |

> **Exam Tip:** DataFrame API = SQL performance (Catalyst Optimizer). Python UDFs are slower than SQL UDFs due to serialization. Higher-order functions preferred over EXPLODE for arrays. `DENSE_RANK` has no gaps, `RANK` has gaps, `ROW_NUMBER` is unique.

---

> **← M05: Incremental Processing | Day 2 | M07: Medallion & Lakeflow →**

## Cleanup

Drop temporary tables, views, and UDFs created during this module. Uncomment the lines below to execute.

---

In [0]:
# Optional cleanup
# Uncomment below to execute cleanup:

# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{GOLD_SCHEMA}.customer_summary")
# spark.sql("DROP FUNCTION IF EXISTS classify_order_sql")
# spark.catalog.clearCache()
# print("Cleanup completed.")